In [ ]:
pip install PyInputPlus

In [ ]:
# 📦 Imports
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pyinputplus as pyip
import re

# 🔧 Style
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 4)


In [ ]:
def get_weather_data(location, forecast_days=3):
    """
    Retrieve weather data from wttr.in for a specified location.

    Args:
        location (str): City or location name
        forecast_days (int): Number of forecast days (1–5)

    Returns:
        dict: Weather data including current conditions and forecast
    """
    if not location or not isinstance(location, str):
        print("❌ Invalid location. Please enter a valid city.")
        return None

    if not (1 <= forecast_days <= 5):
        print("⚠️ Forecast days must be between 1 and 5. Defaulting to 3.")
        forecast_days = 3

    try:
        url = f"https://wttr.in/{location}?format=j1"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()

        if "current_condition" not in data or "weather" not in data:
            print("⚠️ Incomplete weather data received.")
            return None

        data["weather"] = data["weather"][:forecast_days]
        return data

    except requests.exceptions.RequestException as req_err:
        print(f"🔌 Network error: {req_err}")
    except ValueError:
        print("❌ Failed to decode JSON.")
    except Exception as e:
        print(f"⚠️ Unexpected error: {e}")
    return None


In [ ]:
def create_temperature_visualisation(weather_data, output_type='display'):
    try:
        if not weather_data or 'weather' not in weather_data:
            raise ValueError("Invalid or missing weather data.")

        dates = [day['date'] for day in weather_data['weather']]
        max_temps = [int(day['maxtempC']) for day in weather_data['weather']]
        min_temps = [int(day['mintempC']) for day in weather_data['weather']]

        fig, ax = plt.subplots(figsize=(8, 4))
        ax.plot(dates, max_temps, label='Max Temp', marker='o', linewidth=2, color='#FF5733')
        ax.plot(dates, min_temps, label='Min Temp', marker='o', linewidth=2, color='#337DFF')

        ax.set_title("🌡️ Daily Max & Min Temperature Forecast", fontsize=14, weight='bold')
        ax.set_xlabel("Date", fontsize=12)
        ax.set_ylabel("Temperature (°C)", fontsize=12)
        ax.grid(True, linestyle='--', alpha=0.7)
        ax.legend(loc='upper right', fontsize=10)
        ax.tick_params(axis='x', rotation=15)

        for i in range(len(dates)):
            ax.text(dates[i], max_temps[i] + 0.5, f'{max_temps[i]}°', ha='center', fontsize=8, color='#FF5733')
            ax.text(dates[i], min_temps[i] - 2.5, f'{min_temps[i]}°', ha='center', fontsize=8, color='#337DFF')

        plt.tight_layout()
        if output_type == 'display':
            plt.show()
        else:
            return fig

    except Exception as e:
        print(f"❌ Error generating temperature visualisation: {e}")


def create_precipitation_visualisation(weather_data, output_type='display'):
    try:
        if not weather_data or 'weather' not in weather_data:
            raise ValueError("Invalid or missing weather data.")

        dates = [day['date'] for day in weather_data['weather']]
        rain_chances = [int(day['hourly'][0]['chanceofrain']) for day in weather_data['weather']]

        fig, ax = plt.subplots(figsize=(8, 4))
        bars = sns.barplot(x=dates, y=rain_chances, ax=ax, palette='Blues')

        ax.set_title("🌧️ Daily Chance of Rain", fontsize=14, weight='bold')
        ax.set_xlabel("Date", fontsize=12)
        ax.set_ylabel("Chance of Rain (%)", fontsize=12)
        ax.set_ylim(0, 100)
        ax.grid(True, linestyle='--', alpha=0.6)
        ax.tick_params(axis='x', rotation=15)

        for bar, chance in zip(bars.patches, rain_chances):
            ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 1,
                    f'{chance}%', ha='center', fontsize=9)

        plt.tight_layout()
        if output_type == 'display':
            plt.show()
        else:
            return fig

    except Exception as e:
        print(f"❌ Error generating precipitation visualisation: {e}")


In [ ]:
def parse_weather_question(question):
    question = question.lower()
    parsed = {"location": None, "attribute": "general"}

    match = re.search(r'\b(?:in|for)\s+([a-zA-Z\s]+)', question)
    if match:
        parsed["location"] = match.group(1).strip()

    if 'rain' in question:
        parsed["attribute"] = 'rain'
    elif 'temperature' in question or 'hot' in question or 'cold' in question:
        parsed["attribute"] = 'temperature'

    return parsed


def generate_weather_response(parsed_question, weather_data):
    location = parsed_question['location']
    attribute = parsed_question['attribute']
    current = weather_data['current_condition'][0]

    if attribute == 'temperature':
        return (f"🌡️ The current temperature in {location.title()} is "
                f"{current['temp_C']}°C (feels like {current['FeelsLikeC']}°C).")
    elif attribute == 'rain':
        chance = weather_data['weather'][0]['hourly'][0]['chanceofrain']
        return f"🌧️ The chance of rain today in {location.title()} is {chance}%."
    else:
        condition = current['weatherDesc'][0]['value']
        return (f"📍 {location.title()} is currently {condition.lower()} "
                f"with a temperature of {current['temp_C']}°C.")


In [ ]:
def menu():
    print("\n🌤️ Welcome to WeatherWise: Your Intelligent Weather Companion! 🌤️")
    print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")

    while True:
        print("\n🔸 What would you like to do today?")
        choice = pyip.inputMenu(
            ['📍 Check Current Weather',
             '🧠 Ask a Weather Question',
             '📊 View Weather Visualisations',
             '❌ Exit Application'],
            numbered=True
        )

        if choice == '📍 Check Current Weather':
            location = pyip.inputStr("\n🔎 Please enter a city or location: ").strip()
            print("⏳ Retrieving weather data...")
            data = get_weather_data(location)
            if data:
                print("\n✅ Weather Summary for", location.title())
                print("──────────────────────────────────────────")
                print(generate_weather_response({"location": location, "attribute": "general"}, data))
            else:
                print("⚠️ Unable to retrieve weather data.")

        elif choice == '🧠 Ask a Weather Question':
            question = pyip.inputStr("\n💬 Ask your question (e.g., Will it rain in Perth?): ").strip()
            parsed = parse_weather_question(question)
            if parsed['location']:
                print(f"🤖 Analyzing your question about {parsed['location'].title()}...")
                data = get_weather_data(parsed['location


In [ ]:
# Run the app
if __name__ == "__main__":
    menu()

In [3]:
# Manual test: Run this in a separate cell to test visualisation directly
test_data = get_weather_data("Melbourne")
create_temperature_visualisation(test_data)
create_precipitation_visualisation(test_data)

# Manual test: Natural language question
parsed = parse_weather_question("Will it rain in Brisbane?")
if parsed['location']:
    weather = get_weather_data(parsed['location'])
    if weather:
        print(generate_weather_response(parsed, weather))


NameError: name 'get_weather_data' is not defined